# **Mô tả bài toán cho cả 4 thuật toán RF, XGBoost, Ada Boost, Gradient Boost**

Trong bài toán này, chúng ta sẽ được cho một bộ dataset mô tả thông tin về nhân viên trong một công ty, bao gồm các features liên quan đến nhân viên và mức lương của họ. Nhiệm vụ của chúng ta là phân tích, xử lí bộ data dưới đây và trả lời các câu hỏi yêu cầu người làm phải thực hiện coding.  
[employee_data.csv](https://drive.google.com/file/d/1WkVx5tinsxuU3SnIwLNiYU7X2Sqo3vgG/view?usp=sharing)

Tham khảo Link code gốc: [Ada+Gradient](https://drive.google.com/file/d/1wylU7Kro6P5zj1N41zyfLMSWU1qpkrkF/view?usp=sharing)

## **Thực hiện các yêu cầu sau đây**
1. **Đọc dữ liệu**  
Sử dụng pandas, đọc file csv được cung cấp, sau đó hiển thị ra màn hỉnh để hiểu các trường dữ liệu.
2. **Label Encoding**  
Chuyển đổi các cột dữ liệu dạng chữ (cụ thể là cột "Gender" và "Position") sang dạng số bằng cách sử dụng `LabelEncoder` từ thư viện `sklearn`.  
3. **Tách dữ liệu thành bộ feature (X) và label (y)**
- Sử dụng các cột "Gender", "Experience (Years)" và "Position" làm features đầu vào (`X`).
- Sử dụng cột "Salary" làm biến đầu ra (`y`).  
4. **Tách tập dữ liệu thành tập train và test**  
- Chia dữ liệu thành tập huấn luyện (`X_train`, `y_train`) và tập kiểm tra (`X_test`, `y_test`) với tỷ lệ `80:20`.
- Đảm bảo rằng việc chia tách dữ liệu là ngẫu nhiên nhưng tái lập (reproducibility) được với `random_state=42`

# Thư viện

In [1]:
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Tải và đọc dữ liệu

In [2]:
!gdown 1WkVx5tinsxuU3SnIwLNiYU7X2Sqo3vgG

Downloading...
From: https://drive.google.com/uc?id=1WkVx5tinsxuU3SnIwLNiYU7X2Sqo3vgG
To: c:\Users\admin\OneDrive\Máy tính\Exam_AIO2024\AIO_2024_note\Module_3\Source Codes\Ada-Gra Boost\employee_data.csv

  0%|          | 0.00/14.2k [00:00<?, ?B/s]
100%|██████████| 14.2k/14.2k [00:00<00:00, 886kB/s]


In [3]:
# data = pd.read_csv('/content/drive/MyDrive/Exam Module 3/Datasets/RForest, Ada XGBoost /employee_data.csv')
data = pd.read_csv('employee_data.csv')
data.head()

,ID,Gender,Experience (Years),Position,Salary
0,1,F,4,DevOps Engineer,109976
1,2,M,6,DevOps Engineer,120088
2,3,M,17,Web Developer,181301
3,4,M,7,Systems Administrator,77530
4,5,F,13,Systems Administrator,152397


In [4]:
data.describe()

,ID,Experience (Years),Salary
count,400.000000,400.000000,400.00000
mean,200.500000,9.670000,131701.19750
std,115.614301,6.101571,43351.50899
min,1.000000,0.000000,43643.00000
25%,100.750000,4.000000,100484.75000
50%,200.500000,10.000000,128561.50000
75%,300.250000,15.000000,157735.00000
max,400.000000,20.000000,269950.00000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  400 non-null    int64 
 1   Gender              400 non-null    object
 2   Experience (Years)  400 non-null    int64 
 3   Position            400 non-null    object
 4   Salary              400 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 15.8+ KB


# Câu hỏi trắc nghiệm yêu cầu coding

#### Câu 1: Điểm khác biệt chính giữa AdaBoost và Gradient Boosting trong cách mà chúng cải thiện mô hình là gì?

A) AdaBoost tập trung vào việc sửa lỗi của các mẫu dữ liệu có lỗi cao nhất, còn Gradient Boosting tập trung vào giảm thiểu giá trị lỗi toàn bộ bằng cách sử dụng đạo hàm.  
B) AdaBoost sử dụng các mô hình con yếu, trong khi Gradient Boosting chỉ sử dụng mô hình con mạnh.  
C) AdaBoost không thể dẫn đến overfitting, trong khi Gradient Boosting dễ bị overfitting.  
D) AdaBoost và Gradient Boosting có cùng cách tiếp cận trong việc cải thiện mô hình qua các bước lặp.  
Đán án: A

### Câu 2: Điều gì xảy ra khi bạn tăng số lượng mô hình con (estimators) trong AdaBoost hoặc Gradient Boosting? (Thực hiện thay đổi tham số estimators để kiểm tra)

A) Hiệu suất mô hình luôn tăng khi tăng số lượng mô hình con.  
B) Hiệu suất có thể tăng, nhưng nếu quá cao sẽ gây overfitting.  
C) Hiệu suất giảm dần khi tăng số lượng mô hình con.  
D) Hiệu suất không bị ảnh hưởng bởi số lượng mô hình con.  

Đáp án: B

### Huấn luyện mô hình AdaBoost

In [6]:
# Label Encoding for 'Gender' and 'Position'
le_gender = LabelEncoder()
le_position = LabelEncoder()
data['Gender_encoded'] = le_gender.fit_transform(data['Gender'])
data['Position_encoded'] = le_position.fit_transform(data['Position'])

# Prepare features and target
X = data[['Gender_encoded', 'Experience (Years)', 'Position_encoded']]
y = data['Salary']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
	X, y, test_size=0.2, random_state=42
)

ada_regressor = AdaBoostRegressor(n_estimators=50, random_state=42)
ada_regressor.fit(X_train, y_train)

AdaBoostRegressor(random_state=42)

### Huấn luyện mô hình GradientBoost

In [7]:
gb_regressor = GradientBoostingRegressor(n_estimators=50, random_state=42)
gb_regressor.fit(X_train, y_train)

GradientBoostingRegressor(n_estimators=50, random_state=42)

### Câu 3: Khi nào overfitting có thể xảy ra trong AdaBoost và Gradient Boosting? (Hãy thử nghiệm với code các trường hợp dưới đây và đưa ra kết luận)

A) Khi sử dụng quá ít mô hình con.  
B) Khi sử dụng giá trị learning rate quá cao và số lượng mô hình con quá nhiều.  
C) Overfitting không xảy ra trong Gradient Boosting.  
D) Khi mô hình không có đủ dữ liệu để huấn luyện.  
Đáp án: B

### Câu 4: AdaBoost và Gradient Boosting cho phép đánh giá tầm quan trọng của các đặc trưng. Tầm quan trọng của đặc trưng nào sẽ có khả năng cao nhất trong bài toán dự đoán lương nhân viên? (Dùng phương thức feature_importances_ có sẵn trong model)

A) Gender.  
B) Experience (Years).  
C) Position.  
D) ID.  

Đáp án: B

#### Check important feature AdaBoost

In [8]:
importances = ada_regressor.feature_importances_

In [9]:
feature_names = ['Gender', 'Experience (Years)', 'Position']
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

              Feature  Importance
1  Experience (Years)    0.500432
2            Position    0.492506
0              Gender    0.007063


#### Important Features Gradient Boost

In [13]:
importances = gb_regressor.feature_importances_
importances

array([0.00973437, 0.52705277, 0.46321287])

In [11]:
feature_names = ['Gender', 'Experience (Years)', 'Position']
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

              Feature  Importance
1  Experience (Years)    0.527053
2            Position    0.463213
0              Gender    0.009734


### Câu 5: Sử dụng các hàm mean_squared_error, r2_score của thư viện sklearn.metrics để tính toán giá trị MSE và $R^2$ (sử dụng tham số của 2 mô hình là n_estimators=50, random_state=42), trả lời câu hỏi: Dựa trên 2 giá trị trên, hiệu suất của 2 mô hình Ada & Gradient Boost như thế nào?

A. Với giá trị R^2 từ 0.7-0.8, mô hình đang hoạt động khá tốt, giá trị MSE khá tương đối không cao không thấp.

B. Với giá trị R^2 từ 0.6-0.7, mô hình đang hoạt động ở mức khá, bao quát được một phần nhưng cũng nhiều sai sót, giá trị MSE khá cao.

C. VỚi giá trị R2 từ 0.4-0.5, mô hình đang hoạt động ở mức tệ, không nắm bắt được phương sai trong dữ liệu. Giá tri MSE lớn.

D. VỚi giá trị R2 từ 0.1-0.2, mô hình dường như không học được từ dữ liệu. Giá tri MSE siêu lớn thể hiện sự sai sót khi mô hình không thể học.  

Đáp án: B

### Câu 6: (Cho cả 3 phần) Hãy xem đây như là một bài toán thực nghiệm, thay thế từ mô hình Random Forest, XGBoost, AdaBoost, Gradient Boost vào dữ liệu trên. Sau đó đánh giá bộ dữ liệu trên tập test và đưa ra mô hình tốt nhất được chọn tối ưu cho bộ dữ liệu này. (Thực hiện với các siêu tham số chung: n_estimators = 50, random_state=42)

A. Random Forest  
B. XGBoost  
c. AdaBoost  
D. Gradient Boost

Đáp án:

### Compare 4 models

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=50, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=50, random_state=42),
    'AdaBoost': AdaBoostRegressor(n_estimators=50, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=50, random_state=42)
}

# Evaluate models
results = {}

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate R^2 and MSE
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    results[name] = {'R^2': r2, 'MSE': mse}

# Display results
results_df = pd.DataFrame(results).T
print(results_df)

                        R^2           MSE
Random Forest      0.557242  8.270873e+08
XGBoost            0.366861  1.182725e+09
AdaBoost           0.621741  7.066020e+08
Gradient Boosting  0.641661  6.693896e+08
